In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.neighbors import NearestNeighbors 
from scipy.sparse import csr_matrix
import csv

In [7]:
book = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX_Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
rating = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")

In [9]:
book.columns = ['ISBN', 'BookTitle', 'BookAuthor', 'YearOfPublication', 'Publisher', 'ImageUrlS', 'ImageUrlM', 'ImageUrlL']
user.columns = ['UserID', 'Location', 'Age']
rating.columns = ['UserID', 'ISBN', 'BookRating'] 

In [10]:
book

In [11]:
user

In [12]:
rating

In [13]:
c1_usid = rating['UserID'].value_counts()  
c1_usid

In [14]:
rating = rating[rating['UserID'].isin(c1_usid[c1_usid >= 10].index)]
rating['UserID'].value_counts()

In [15]:
c2_usid = rating['BookRating'].value_counts()  
c2_usid

In [16]:
rating = rating[rating['BookRating'].isin(c2_usid[c2_usid >= 10].index)]
c1 = rating['BookRating'].value_counts()
c1

In [17]:
book_rating = pd.merge(rating, book, on='ISBN')
book_rating

In [18]:
columns = ['BookAuthor', 'YearOfPublication', 'Publisher', 'ImageUrlS', 'ImageUrlM', 'ImageUrlL']
book_rating = book_rating.drop(columns, axis=1)
book_rating

In [21]:
book_rating = book_rating.dropna()
book_rating

In [25]:
bkrating_cnt = (book_rating.groupby(by = ['BookTitle'])['BookRating'].count().reset_index().rename(
columns = {'BookRating': 'TotalRating'})[['BookTitle', 'TotalRating']]) 
bkrating_cnt

In [28]:
rating_bkrating_cnt = book_rating.merge(bkrating_cnt, left_on = 'BookTitle', right_on = 'BookTitle', how = 'left')
rating_bkrating_cnt

In [33]:
a = rating_bkrating_cnt['TotalRating']
a[a>=10].value_counts()

In [35]:
a[a>=10]

In [34]:
b=rating_bkrating_cnt[rating_bkrating_cnt['TotalRating']>=10]
b

In [52]:
user.isna().sum()

In [60]:
user.drop('Age',axis=1,inplace=True)
user.drop('Location',axis=1,inplace=True)
user

In [61]:
final = b.merge(user,left_on = 'UserID', right_on = 'UserID', how = 'left')
final

In [66]:
final = final.drop_duplicates(['UserID', 'BookTitle'])
final

In [67]:
final.to_csv('Final.csv')

In [69]:
final_pivot = final.pivot(index = 'BookTitle', columns = 'UserID', values = 'BookRating').fillna(0)
final_matrix = csr_matrix(final_pivot.values)

In [70]:
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(final_matrix)

In [71]:
finalmodel = 'Final_Model.sav'
pickle.dump(model, open(finalmodel, 'wb'))